# IMPORTING THE LIBRARIES 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
import os

###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl



from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# DATA ARRANGEMENT

In the main directory we can see that there are 8 subfolders. 

Each subfolder have almost 12 spectras per sample , the idea behind that would be , instead of having just one spectra per sample , and to just rely on one information , its always better have to multiple measurements per samples , and then this could be used for building the Calibration Model

Instead of having 12 different csv per samples , its always good to have a single dataframe -> This new dataframe will have 1st column as wavelength , and 2nd -13th column as Intensities 

In [3]:
def load_data(folder_path):
    # List to store DataFrames for intensity columns
    intensity_dfs = []

    # List to store CSV file names
    csv_file_names = []

    # Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            
            # Read CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Extract intensity column and store it in the list
            intensity_dfs.append(df.iloc[:, 1])  # Assuming intensity is in the second column
            
            # Store CSV file name
            csv_file_names.append(os.path.splitext(file_name)[0])

    # Read the wavelength column from the first CSV file
    wavelength_df = pd.read_csv(os.path.join(folder_path, os.listdir(folder_path)[1]), usecols=[0])

    # Concatenate wavelength column with intensity columns
    result_df = pd.concat([wavelength_df] + intensity_dfs, axis=1)

    # Rename the columns with CSV file names
    column_names = ['wavelength'] + [f'{csv_file_names[i]}' for i in range(len(intensity_dfs))]

    result_df.columns = column_names
    # result_df.reset_index(drop=True, inplace=True)

    return result_df

In [4]:
def baseline_correction(df):
    """
    Perform baseline correction on the intensity columns of the input DataFrame and create a new DataFrame with corrected values.
    
    Parameters:
        df (DataFrame): Input DataFrame containing the wavelength and intensity columns.
        
    Returns:
        DataFrame: New DataFrame with baseline-corrected intensity columns and the same wavelength column as the input DataFrame.
    """
    # Copy the 'wavelength' column from the input DataFrame
    new_df = pd.DataFrame({'wavelength': df['wavelength']})
    
    # Perform baseline correction for each intensity column and add them to the new DataFrame
    for col in df.columns[1:]:  # Exclude the 'wavelength' column
        baseline, _ = pl.airpls(df[col],lam=0.1)
        corrected_values = df[col] - baseline
        new_df[col] = corrected_values
    
    return new_df

Loading the dataframe.

In [5]:
CR210LA_Raw_df = load_data('CR210LA')
###################################################
CR240LA_Raw_df = load_data('CR240LA')
###################################################
CR300LA_Raw_df = load_data('CR300LA')
###################################################
CR440Y_Raw_df = load_data('CR440Y')
###################################################
CR570CP_Raw_df = load_data('CR570CP')
###################################################
CR700Y_Raw_df = load_data('CR700Y')
###################################################
HR660Y_Raw_df = load_data('HR660Y')
# ###################################################
CR1000Y_Raw_df = load_data('CR1000Y')

In [6]:
CR210LA_BaselineCorrected_df = baseline_correction(CR210LA_Raw_df)

CR240LA_BaselineCorrected_df = baseline_correction(CR240LA_Raw_df)

CR300LA_BaselineCorrected_df = baseline_correction(CR300LA_Raw_df)

CR440Y_BaselineCorrected_df = baseline_correction(CR440Y_Raw_df)

CR570CP_BaselineCorrected_df = baseline_correction(CR570CP_Raw_df)

CR700Y_BaselineCorrected_df = baseline_correction(CR700Y_Raw_df)

HR660Y_BaselineCorrected_df = baseline_correction(HR660Y_Raw_df)

CR1000Y_BaselineCorrected_df = baseline_correction(CR1000Y_Raw_df)

# Peak Selection and Data Trimming

The dataframe  which we have is very big ,it could be trimmed now according to the wavelength , by adjusting two parameters "Wavelength_Min" , "Wavelength_Max"

In [7]:
Wavelength_Min = 202.15
Wavelength_Max = 203.1

CR210LA_Select_df = CR210LA_BaselineCorrected_df[(CR210LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR210LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR240LA_Select_df = CR240LA_BaselineCorrected_df[(CR240LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR240LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR300LA_Select_df = CR300LA_BaselineCorrected_df[(CR300LA_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR300LA_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR440Y_Select_df = CR440Y_BaselineCorrected_df[(CR440Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR440Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR570CP_Select_df = CR570CP_BaselineCorrected_df[(CR570CP_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR570CP_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR700Y_Select_df = CR700Y_BaselineCorrected_df[(CR700Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR700Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
HR660Y_Select_df = HR660Y_BaselineCorrected_df[(HR660Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (HR660Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]
CR1000Y_Select_df = CR1000Y_BaselineCorrected_df[(CR1000Y_BaselineCorrected_df['wavelength'] >= Wavelength_Min) & (CR1000Y_BaselineCorrected_df['wavelength'] <= Wavelength_Max)]

CR210LA_Select_df


,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
665,202.166667,2739.695591,1395.888460,100.032479,1328.481918,1804.948596,1170.967817,4.484428,1516.589577,1903.273104,2.947669,1887.476828,1863.354250
666,202.200000,3193.308542,1948.531006,434.263733,1932.530291,2368.652903,1549.137240,237.366308,1925.880502,2322.432620,216.207732,2283.673899,2303.780836
667,202.233333,3822.239690,2614.810731,986.705680,2626.723462,3024.916657,2126.684680,669.300995,2541.028471,2938.673363,610.218129,2878.837854,2960.479496
668,202.266667,4730.791968,3506.379828,1679.068314,3511.737742,3889.533481,2817.405579,1208.758914,3274.403399,3664.612576,1099.626398,3581.377678,3745.150079
669,202.300000,6054.775234,4927.252568,2489.808639,4909.989875,5275.131636,3592.564766,1826.612318,4099.061550,4475.793001,1662.271802,4359.709793,4628.428541
670,202.333333,7929.742567,7186.661758,3712.408462,7151.907428,7500.683724,4742.637594,2840.196594,5325.015734,5696.917888,2626.354398,5515.547332,5908.363688
671,202.366667,10401.648023,10240.533254,5748.144246,10217.612423,10552.315542,6667.327413,4685.400244,7377.578539,7772.166354,4439.961303,7464.651277,7995.038408
672,202.400000,13415.952447,13648.527970,8770.361343,13694.328286,14042.835805,9565.807768,7574.278975,10454.076812,10913.346585,7331.322415,10408.299578,11076.180073
673,202.433333,16894.549456,16957.551906,12128.691978,17152.707994,17567.660111,12912.518200,10909.945888,13927.903141,14492.478337,10734.082093,13772.616668,14535.930578
674,202.466667,20482.045553,19839.899121,14987.823748,20228.078337,20758.594689,16018.875735,13913.689947,16987.335553,17692.843862,13903.174467,16809.489338,17577.928525


In [8]:
CR210LA_Select_Raw_df = CR210LA_Raw_df[(CR210LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR210LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR240LA_Select_Raw_df = CR240LA_Raw_df[(CR240LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR240LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR300LA_Select_Raw_df = CR300LA_Raw_df[(CR300LA_Raw_df['wavelength'] >= Wavelength_Min) & (CR300LA_Raw_df['wavelength'] <= Wavelength_Max)]
CR440Y_Select_Raw_df = CR440Y_Raw_df[(CR440Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR440Y_Raw_df['wavelength'] <= Wavelength_Max)]
CR570CP_Select_Raw_df = CR570CP_Raw_df[(CR570CP_Raw_df['wavelength'] >= Wavelength_Min) & (CR570CP_Raw_df['wavelength'] <= Wavelength_Max)]
CR700Y_Select_Raw_df = CR700Y_Raw_df[(CR700Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR700Y_Raw_df['wavelength'] <= Wavelength_Max)]
HR660Y_Select_Raw_df = HR660Y_Raw_df[(HR660Y_Raw_df['wavelength'] >= Wavelength_Min) & (HR660Y_Raw_df['wavelength'] <= Wavelength_Max)]
CR1000Y_Select_Raw_df = CR1000Y_Raw_df[(CR1000Y_Raw_df['wavelength'] >= Wavelength_Min) & (CR1000Y_Raw_df['wavelength'] <= Wavelength_Max)]

CR210LA_Select_Raw_df

,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
665,202.166667,3364.614875,3864.423606,2452.816479,3876.268310,3759.857255,2140.434378,2127.833692,2234.583455,2182.162847,2043.497661,2135.632526,2432.109944
666,202.200000,3824.006769,4524.889663,2927.468415,4573.492982,4359.126010,2535.740775,2526.233444,2653.650614,2603.911206,2456.339939,2530.314392,2873.306398
667,202.233333,4458.673998,5297.455347,3616.251612,5357.443087,5047.820574,3129.888445,3121.314779,3278.335265,3222.773666,3042.964378,3123.961542,3530.601359
668,202.266667,5372.917206,6293.190532,4440.062945,6328.524379,5941.789375,3836.634334,3820.251748,4020.992865,3951.366120,3716.957912,3824.987836,4315.705609
669,202.300000,6702.543963,7815.527173,5376.546303,7808.883044,7353.715483,4627.204777,4592.620231,4854.665221,4765.229960,4455.099202,4601.814574,5199.266044
670,202.333333,8583.105057,10173.115761,6718.370382,10128.676092,9602.626307,5792.036631,5754.459237,6089.350684,5989.067083,5584.527700,5756.159765,6479.342402
671,202.366667,11060.552257,13321.299837,8865.998524,13267.754990,12674.702452,7730.794752,7740.360898,8150.347385,8067.055259,7552.269920,7703.789270,8566.028510
672,202.400000,14080.344118,16819.157997,11991.962970,16813.072606,16182.807440,10642.614188,10761.084542,11234.967715,11211.001322,10585.495159,10645.985913,11647.062674
673,202.433333,17564.371969,20213.013926,15445.082828,20335.011359,19722.411677,14001.895985,14218.446901,14716.589803,14792.923677,14116.787170,14008.879004,15106.597727
674,202.466667,21157.240023,23174.579366,18389.232580,23468.627484,22925.376201,17120.018677,17332.440560,17783.477217,17996.103226,17400.019480,17044.360212,18148.283205


Lets plot a line plot ,to get a better picture 

In [9]:
Selected_df_Plot = figure(title = 'Selected Data Plot' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_1 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_2 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_3 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_4 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_5 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_6 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_7 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_8 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_9 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_10 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_11 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_12 , line_width = 2, color ="red" )

######################################################################################################################
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_1 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_2 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_3 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_4 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_5 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_6 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_7 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_8 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_9 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_10 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_11 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_12 , line_width = 2, color ="green" )

###############################################################################################################

Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_1 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_2 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_3 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_4 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_5 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_6 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_7 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_8 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_9 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_10 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_11, line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_12, line_width = 2, color ="pink")

######################################################################################################################

Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_1 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_2 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_3 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_4 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_5 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_6 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_7 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_8 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_9 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_10 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_11, line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_12, line_width = 2, color ="yellow")

##############################################################################################################

Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_1 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_2 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_3 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_4 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_5 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_6 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_7 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_8 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_9 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_10 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_11, line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_12, line_width = 2, color ="blue")

###############################################################################################################

Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_1 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_2 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_3 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_4 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_5 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_6 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_7 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_8 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_9 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_10 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_11, line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_12, line_width = 2, color ="orange")


####################################################################################################################

Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_1 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_2 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_3 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_4 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_5 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_6 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_7 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_8 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_9 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_10 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_11 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_12 , line_width = 2, color ="brown")


##################################################################################################################

Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_1 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_2 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_3 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_4 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_5 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_6 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_7 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_8 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_9 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_10 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_11 , line_width = 2, color ="black")
Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_12 , line_width = 2, color ="black")

###################################################################################################################

Selected_df_Plot.width = 600
Selected_df_Plot.height = 500
show(Selected_df_Plot)

# Data Preprocessing of the Spectra

The above plot eventhough a spectra , is still a Raw Spectra , which still has lot of Artifects , before proceeding for the Univariate Calibration , its important to Pre Process the Raw Spectra accordingly. Various Pre Processing Techniques could be used here :- 

1) Baseline Correction - Very Very little  background radiation is still present in the spectra, which corresponds to the spectral baseline and imposes difficulties for quantitative elemental analysis.

In [10]:
Baseline_Correction_Plot = figure(title = 'Baseline Correction' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Baseline_Correction_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_1 , line_width = 2, color ="green" )
Baseline_Correction_Plot.line(CR210LA_Select_Raw_df.wavelength,CR210LA_Select_Raw_df.CR210LA_1 , line_width =2 , color = "red")
Baseline_Correction_Plot.width = 600
Baseline_Correction_Plot.height = 500
show(Baseline_Correction_Plot)

2) Normalization - Its usually noticed that , for a measurement of a similar sample , the spectra obtained for them are quite different , but there could me many reasons for it , like laser energy fluctuations , material surface , even though is homegnous isnt same overall the surface

   For Instance , you could see the plot below , Though this are the plots from the same sample CR210LA , measured on 12 different Areas , its quite visible , for some  spectras , the peak heights or Intensities are not same.


In [11]:

Before_Normalization = figure(title = 'Before Normalization' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_1 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_2 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_3 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_4 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_5 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_6 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_7 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_8 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_9 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_10 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_11 , line_width = 2, color ="red" )
Before_Normalization.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_12 , line_width = 2, color ="red" )


Before_Normalization.width = 600
Before_Normalization.height = 500
show(Before_Normalization)

Now , its important the understand this chnages in Peaks , are common not just to a single peaks , but to the entire spectra. If theres a change in intensity of a peak , its not just for a single peak , but it applies to the entire spectrum proportionally. 

Normalization by peak of an Matrix Element i.e Fe 238.20 (237.9 ~ 238.7) , 263.10 (262.9 ~ 263.4)

Lets try to focus now on analysis for peak of Matrix Element

In [12]:
Matrix_Wavelength_Min = 262.9
Matrix_Wavelength_Max = 263.4

CR210LA_Matrix_df = CR210LA_BaselineCorrected_df[(CR210LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR210LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR240LA_Matrix_df = CR240LA_BaselineCorrected_df[(CR240LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR240LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR300LA_Matrix_df = CR300LA_BaselineCorrected_df[(CR300LA_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR300LA_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR440Y_Matrix_df = CR440Y_BaselineCorrected_df[(CR440Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR440Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR570CP_Matrix_df = CR570CP_BaselineCorrected_df[(CR570CP_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR570CP_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR700Y_Matrix_df = CR700Y_BaselineCorrected_df[(CR700Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR700Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
HR660Y_Matrix_df = HR660Y_BaselineCorrected_df[(HR660Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (HR660Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]
CR1000Y_Matrix_df = CR1000Y_BaselineCorrected_df[(CR1000Y_BaselineCorrected_df['wavelength'] >= Matrix_Wavelength_Min) & (CR1000Y_BaselineCorrected_df['wavelength'] <= Matrix_Wavelength_Max)]

CR210LA_Matrix_df

,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
2487,262.900000,5244.341280,2310.343671,2948.382699,1491.142851,2500.199641,3571.778812,2829.251338,2460.656198,2527.833464,2170.980312,2586.561581,2686.057220
2488,262.933333,4245.604653,1422.598203,3088.860776,796.067571,1698.185890,3730.491359,3019.069377,2667.912693,2700.663174,2200.026177,2713.658795,2664.202073
2489,262.966667,6258.440092,2994.723336,4586.330270,1733.118569,3406.458417,5265.117220,4662.356916,4235.642510,4282.121667,3363.646767,4377.883174,3991.367265
2490,263.000000,11888.500891,7681.192527,7038.971716,4792.767009,8210.430281,7805.184975,7365.075380,6766.822169,6879.923898,5439.627068,7198.262250,6477.844794
2491,263.033333,18521.451221,13213.917277,9898.181854,8509.635260,13818.502641,10834.715049,10578.958285,9718.665945,9951.842492,8089.077018,10632.970914,9799.160161
2492,263.066667,22394.961331,16282.913848,12272.955837,10720.913894,16913.491012,13414.588934,13324.777218,12188.602439,12558.832871,10513.307644,13634.812900,12965.091292
2493,263.100000,21323.680018,15030.684964,12848.117160,10175.321223,15633.257467,14081.497962,14089.422379,12828.352962,13270.268006,11344.024954,14530.531420,14160.679142
2494,263.133333,17023.974728,11193.388510,10492.503334,7876.959479,11715.596437,11569.787326,11564.559418,10476.932279,10851.197455,9384.623541,11868.182128,11788.150372
2495,263.166667,11778.766874,7019.629477,6225.282956,5155.467804,7412.385390,6967.092587,6874.092679,6170.888048,6379.034749,5555.244527,6855.650616,6993.604024
2496,263.200000,7106.832100,3776.385691,2324.469395,2840.715436,4014.115032,2738.710312,2565.805372,2233.245543,2284.895335,2015.212767,2257.937587,2529.454292


In [13]:
Selected_Matrix_Plot = figure(title = 'Selected Matrix Peak' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_1 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_2 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_3 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_4 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_5 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_6 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_7 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_8 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_9 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_10 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_11 , line_width = 2, color ="red" )
Selected_Matrix_Plot.line(CR210LA_Matrix_df.wavelength,CR210LA_Matrix_df.CR210LA_12 , line_width = 2, color ="red" )

######################################################################################################################
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_1 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_2 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_3 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_4 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_5 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_6 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_7 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_8 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_9 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_10 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_11 , line_width = 2, color ="green" )
Selected_Matrix_Plot.line(CR240LA_Matrix_df.wavelength,CR240LA_Matrix_df.CR240LA_12 , line_width = 2, color ="green" )

###############################################################################################################

Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_1 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_2 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_3 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_4 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_5 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_6 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_7 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_8 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_9 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_10 , line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_11, line_width = 2, color ="pink")
Selected_Matrix_Plot.line(CR300LA_Matrix_df.wavelength,CR300LA_Matrix_df.CR300LA_12, line_width = 2, color ="pink")

######################################################################################################################

Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_1 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_2 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_3 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_4 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_5 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_6 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_7 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_8 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_9 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_10 , line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_11, line_width = 2, color ="yellow")
Selected_Matrix_Plot.line(CR440Y_Matrix_df.wavelength,CR440Y_Matrix_df.CR440Y_12, line_width = 2, color ="yellow")

##############################################################################################################

Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_1 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_2 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_3 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_4 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_5 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_6 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_7 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_8 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_9 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_10 , line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_11, line_width = 2, color ="blue")
Selected_Matrix_Plot.line(CR570CP_Matrix_df.wavelength,CR570CP_Matrix_df.CR570CP_12, line_width = 2, color ="blue")

###############################################################################################################

Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_1 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_2 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_3 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_4 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_5 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_6 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_7 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_8 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_9 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_10 , line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_11, line_width = 2, color ="orange")
Selected_Matrix_Plot.line(CR700Y_Matrix_df.wavelength,CR700Y_Matrix_df.CR700Y_12, line_width = 2, color ="orange")


####################################################################################################################

Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_1 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_2 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_3 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_4 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_5 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_6 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_7 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_8 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_9 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_10 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_11 , line_width = 2, color ="brown")
Selected_Matrix_Plot.line(HR660Y_Matrix_df.wavelength,HR660Y_Matrix_df.HR660Y_12 , line_width = 2, color ="brown")


##################################################################################################################

Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_1 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_2 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_3 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_4 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_5 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_6 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_7 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_8 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_9 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_10 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_11 , line_width = 2, color ="black")
Selected_Matrix_Plot.line(CR1000Y_Matrix_df.wavelength,CR1000Y_Matrix_df.CR1000Y_12 , line_width = 2, color ="black")

###################################################################################################################

Selected_Matrix_Plot.width = 600
Selected_Matrix_Plot.height = 500
show(Selected_Matrix_Plot)

In [14]:

def normalize_data(matrix_df, baseline_df):
    normalized_df = pd.DataFrame() #crete a df
    normalized_df['wavelength'] = baseline_df['wavelength'] #add first column as wavelength

    for sample_col in matrix_df.columns[1:]:
        peaks, _ = fp(matrix_df[sample_col], prominence=1000) #finds peak from matrix_df
        return_intensities = matrix_df[sample_col].iloc[peaks] #gets the intensity for that matrix peak
        # return_intensities = max(matrix_df[sample_col]]      #Normalization by Maximum Intensity
        corresponding_col = sample_col
        normalized_df[sample_col] = baseline_df[corresponding_col] / return_intensities.values[0] #create a columns in normalized_df , by dividing the baseline_df by the intensities of the matrix peak 
    
    return normalized_df


In [15]:
# normalized_df = pd.DataFrame()
# normalized_df['wavelength'] = CR210LA_BaselineCorrected_df['wavelength']

# for sample_col in CR210LA_Matrix_df.columns[1:]:
#         peaks, _ = fp(CR210LA_Matrix_df[sample_col], prominence=1000 )
#         return_intensities = CR210LA_Matrix_df[sample_col].iloc[peaks]
#         correspondin_col = sample_col
#         print(return_intensities)
#         print(correspondin_col)
#         normalized_df[sample_col] = CR210LA_BaselineCorrected_df[correspondin_col] / return_intensities.values[0] 

# print (normalized_df)

In [16]:
CR210LA_Normalized_df = normalize_data (matrix_df=CR210LA_Matrix_df , baseline_df=CR210LA_BaselineCorrected_df)
CR240LA_Normalized_df = normalize_data (matrix_df=CR240LA_Matrix_df , baseline_df=CR240LA_BaselineCorrected_df)
CR300LA_Normalized_df = normalize_data (matrix_df=CR300LA_Matrix_df , baseline_df=CR300LA_BaselineCorrected_df)
CR440Y_Normalized_df = normalize_data (matrix_df=CR440Y_Matrix_df , baseline_df=CR440Y_BaselineCorrected_df)
CR570CP_Normalized_df = normalize_data (matrix_df=CR570CP_Matrix_df , baseline_df=CR570CP_BaselineCorrected_df)
CR700Y_Normalized_df = normalize_data (matrix_df=CR700Y_Matrix_df , baseline_df=CR700Y_BaselineCorrected_df)
HR660Y_Normalized_df = normalize_data (matrix_df=HR660Y_Matrix_df , baseline_df=HR660Y_BaselineCorrected_df)
CR1000Y_Normalized_df = normalize_data (matrix_df=CR1000Y_Matrix_df , baseline_df=CR1000Y_BaselineCorrected_df)
# print (CR210LA_Normalized_df.iloc[2493])

In [19]:
CR210LA_Processed_Select_df = CR210LA_Normalized_df[(CR210LA_Normalized_df['wavelength'] >= Wavelength_Min) & (CR210LA_Normalized_df['wavelength'] <= Wavelength_Max)]
CR240LA_Processed_Select_df = CR240LA_Normalized_df[(CR240LA_Normalized_df['wavelength'] >= Wavelength_Min) & (CR240LA_Normalized_df['wavelength'] <= Wavelength_Max)]
CR300LA_Processed_Select_df = CR300LA_Normalized_df[(CR300LA_Normalized_df['wavelength'] >= Wavelength_Min) & (CR300LA_Normalized_df['wavelength'] <= Wavelength_Max)]
CR440Y_Processed_Select_df = CR440Y_Normalized_df[(CR440Y_Normalized_df['wavelength'] >= Wavelength_Min) & (CR440Y_Normalized_df['wavelength'] <= Wavelength_Max)]
CR570CP_Processed_Select_df = CR570CP_Normalized_df[(CR570CP_Normalized_df['wavelength'] >= Wavelength_Min) & (CR570CP_Normalized_df['wavelength'] <= Wavelength_Max)]
CR700Y_Processed_Select_df = CR700Y_Normalized_df[(CR700Y_Normalized_df['wavelength'] >= Wavelength_Min) & (CR700Y_Normalized_df['wavelength'] <= Wavelength_Max)]
HR660Y_Processed_Select_df = HR660Y_Normalized_df[(HR660Y_Normalized_df['wavelength'] >= Wavelength_Min) & (HR660Y_Normalized_df['wavelength'] <= Wavelength_Max)]
CR1000Y_Processed_Select_df = CR1000Y_Normalized_df[(CR1000Y_Normalized_df['wavelength'] >= Wavelength_Min) & (CR1000Y_Normalized_df['wavelength'] <= Wavelength_Max)]

CR210LA_Processed_Select_df

,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
665,202.166667,0.122335,0.085727,0.007786,0.123915,0.106717,0.083156,0.000318,0.118222,0.143424,0.000260,0.129897,0.131587
666,202.200000,0.142590,0.119667,0.033800,0.180258,0.140045,0.110012,0.016847,0.150127,0.175010,0.019059,0.157164,0.162689
667,202.233333,0.170674,0.160586,0.076798,0.245009,0.178846,0.151027,0.047504,0.198079,0.221448,0.053792,0.198123,0.209063
668,202.266667,0.211244,0.215341,0.130686,0.327560,0.229966,0.200079,0.085792,0.255247,0.276152,0.096934,0.246473,0.264475
669,202.300000,0.270363,0.302603,0.193788,0.457982,0.311889,0.255127,0.129644,0.319531,0.337280,0.146533,0.300038,0.326851
670,202.333333,0.354086,0.441362,0.288946,0.667099,0.443473,0.336799,0.201584,0.415097,0.429299,0.231519,0.379583,0.417237
671,202.366667,0.464464,0.628913,0.447392,0.953054,0.623899,0.473481,0.332547,0.575099,0.585683,0.391392,0.513722,0.564594
672,202.400000,0.599061,0.838212,0.682618,1.277347,0.830274,0.679317,0.537586,0.814920,0.822391,0.646272,0.716306,0.782179
673,202.433333,0.754391,1.041432,0.944005,1.599930,1.038677,0.916985,0.774336,1.085712,1.092101,0.946232,0.947840,1.026500
674,202.466667,0.914583,1.218449,1.166539,1.886787,1.227339,1.137583,0.987527,1.324202,1.333270,1.225594,1.156839,1.241320


In [20]:

After_Normalization = figure(title = 'After Normalization' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_1 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_2 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_3 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_4 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_5 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_6 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_7 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_8 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_9 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_10 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_11 , line_width = 2, color ="red" )
After_Normalization.line(CR300LA_Processed_Select_df.wavelength,CR300LA_Processed_Select_df.CR300LA_12 , line_width = 2, color ="red" )


After_Normalization.width = 600
After_Normalization.height = 500
show(After_Normalization)

# OUTLIER REMOVAL 

As you see from the Pic above , even after normalizing the Spectra , there are few variations in the Intensities , Before passing them to regression step , the important step would be to do Outlier Removal of the Intensities

In [27]:
Intensities_Selected = []
all_intensities = []

for col in CR210LA_Processed_Select_df.columns[1:]:
        peak, _ = fp(CR210LA_Processed_Select_df[col], prominence=0.1 )
        all_intensities = CR210LA_Processed_Select_df[col].iloc[peak]
        print(all_intensities)



677    1.190078
Name: CR210LA_10, dtype: float64
677    1.46996
Name: CR210LA_11, dtype: float64
677    1.417407
Name: CR210LA_8, dtype: float64
677    2.28611
Name: CR210LA_9, dtype: float64
677    1.49926
Name: CR210LA_12, dtype: float64
677    1.453147
Name: CR210LA_4, dtype: float64
677    1.266791
Name: CR210LA_5, dtype: float64
677    1.623779
Name: CR210LA_7, dtype: float64
677    1.646938
Name: CR210LA_6, dtype: float64
677    1.63015
Name: CR210LA_2, dtype: float64
677    1.440446
Name: CR210LA_3, dtype: float64
678    1.529429
Name: CR210LA_1, dtype: float64
